# 4-fold Regression Cross Validation

In [1]:
import tensorflow as tf
import os
cwd = os.getcwd()
os.chdir('../../src/')
from dataloader import OneHotEncoder, load_fasta_data, RNASeqDataGenerator
from regression import seq2twohot
os.chdir('../experiments/CrossValidation/')

In [2]:
def rmse(x, y):
    return np.sqrt(np.mean((x-y)**2))

In [3]:
from tqdm import tqdm
be_type='CBE'
sample='162F'
be_class='A3A'

data_dir = "../../data/raw/{0}/{0}-sequence/".format(be_type)

train_chr = [str(x) for x in range(1, 23)]+['X']

train_seqs, train_ers = [], []
for rep in tqdm(train_chr):
        seq, er = load_fasta_data(sample, rep, data_dir)
        train_seqs = seq+train_seqs
        train_ers = er+train_ers
splits = len(train_seqs)//4

100%|██████████| 23/23 [00:09<00:00,  2.46it/s]


In [4]:
import numpy as np 
from sklearn.linear_model import Ridge
from scipy import stats

p = []
s = []
r = []

for fold in range(1, 5):
    fold_test_seqs = train_seqs[splits*(fold-1):splits*fold]
    fold_test_ers = train_ers[splits*(fold-1):splits*fold]
    fold_train_seqs = train_seqs[0:splits*(fold-1)] + train_seqs[splits*fold:]
    fold_train_ers = train_ers[0:splits*(fold-1)]  + train_ers[splits*fold:]
    train_seq_reg = np.array([seq2twohot(x) for x in fold_train_seqs])
    test_seq_reg = np.array([seq2twohot(x) for x in fold_test_seqs])
    reg = Ridge().fit(train_seq_reg, fold_train_ers)
    preds = reg.predict(test_seq_reg)
    p.append(stats.pearsonr(preds, fold_test_ers)[0])
    s.append(stats.spearmanr(preds, fold_test_ers)[0])
    r.append(rmse(preds, fold_test_ers))

In [5]:
import pandas as pd
df = pd.DataFrame({'Fold':range(1,5), 'RMSE':r, 'SpearmanR':s, 'PearsonR':p})

In [6]:
df.to_csv("{}/{}REGresults.csv".format(be_class, sample),index=False)